In [10]:
import requests
import PyPDF2
from io import BytesIO

In [3]:

def get_pmc_data(id=None, from_date=None, until=None, format=None, resumption_token=None):
    base_url = "https://www.ncbi.nlm.nih.gov/pmc/utils/oa/oa.fcgi"
    params = {}

    if id:
        params['id'] = id
    if from_date:
        params['from'] = from_date
    if until:
        params['until'] = until
    if format:
        params['format'] = format
    if resumption_token:
        params['resumptionToken'] = resumption_token

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        return response.text
    else:
        return None

In [5]:
import xml.etree.ElementTree as ET

xml_data = get_pmc_data(id='PMC5334499')

root = ET.fromstring(response.text)

# Get response date
response_date = root.find('responseDate').text
print(f"Response Date: {response_date}")

# Get request id
request_id = root.find('request').attrib['id']
print(f"Request ID: {request_id}")

# Get record details
for record in root.iter('record'):
    record_id = record.attrib['id']
    citation = record.attrib['citation']
    license = record.attrib['license']
    retracted = record.attrib['retracted']
    print(f"Record ID: {record_id}, Citation: {citation}, License: {license}, Retracted: {retracted}")

    # Get link details
    for link in record.iter('link'):
        format = link.attrib['format']
        updated = link.attrib['updated']
        href = link.attrib['href']
        print(f"Link Format: {format}, Updated: {updated}, Href: {href}")

Response Date: 2023-11-10 13:30:55
Request ID: PMC5334499
Record ID: PMC5334499, Citation: World J Radiol. 2017 Feb 28; 9(2):27-33, License: CC BY-NC, Retracted: no
Link Format: tgz, Updated: 2021-12-16 16:16:38, Href: ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_package/8e/71/PMC5334499.tar.gz
Link Format: pdf, Updated: 2017-03-03 06:05:17, Href: ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_pdf/8e/71/WJR-9-27.PMC5334499.pdf


In [18]:
import urllib.request
import PyPDF2
from io import BytesIO

def scrape_text_from_pdf_url(url):
    response = urllib.request.urlopen(url)
    file = BytesIO(response.read())
    reader = PyPDF2.PdfReader(file)
    text = ''
    for page in reader.pages:
        text += page.extract_text()
    return text

In [19]:

text = scrape_text_from_pdf_url(href)
print(text)

Matteo Bauckneht, Roberta Piva, Gianmario Sambuceti, Francesco Grossi, Silvia Morbelli EDITORIAL
27 February 28, 2017 |Volume 9 |Issue 2| WJR|www.wjgnet.comEvaluation of response to immune checkpoint inhibitors: Is 
there a role for positron emission tomography?
Matteo Bauckneht, Roberta Piva, Gianmario Sambuceti, 
Silvia Morbelli, Nuclear Medicine Unit, IRCCS San Martino-
IST, University of Genoa, 16132 Genoa, Italy
Francesco Grossi,  Lung Cancer Unit, IRCCS San Martino-IST, 
University of Genoa, 16132 Genoa, Italy
Author contributions:  Morbelli S conceived and designed 
the study; Bauckneht M and Morbelli S drafted the manuscript; 
Bauckneht M and Piva R prepared the tables and figures; 
Sambuceti G and Grossi F critically revised the manuscript; all 
the authors approved the final version of the paper.
Conflict-of-interest statement:  The authors have no conflicts of 
interest related to this publication to disclose.
Open-Access:  This article is an open-access article which was 
s